In [1]:
import tkinter as tk
from tkinter import messagebox, simpledialog
import time

class KnightTour:
    def __init__(self, n):
        self.n = n
        self.path_row = [2, 1, -1, -2, -2, -1, 1, 2]
        self.path_col = [1, 2, 2, 1, -1, -2, -2, -1]
        self.moves = []  # To store the sequence of moves

    def find_knight_tour(self, visited, row, col, move, canvas, cell_size):
        if move == self.n * self.n + 1:  # Base case when all cells are visited
            return True
        else:
            # Use Warnsdorff's heuristic to prioritize moves
            next_moves = self.get_sorted_moves(visited, row, col)
            for row_new, col_new in next_moves:
                visited[row_new][col_new] = move
                self.moves.append((row_new, col_new))
                self.update_canvas(canvas, visited, cell_size)
                time.sleep(0.2)  # Slight delay to show trying solutions
                if self.find_knight_tour(visited, row_new, col_new, move + 1, canvas, cell_size):
                    return True
                visited[row_new][col_new] = 0  # Backtrack
                self.moves.pop()
        return False

    def get_sorted_moves(self, visited, row, col):
        moves = []
        for index in range(len(self.path_row)):
            row_new = row + self.path_row[index]
            col_new = col + self.path_col[index]
            if self.if_valid_move(visited, row_new, col_new):
                degree = self.get_degree(visited, row_new, col_new)
                moves.append((degree, row_new, col_new))
        moves.sort()  # Sort by degree (Warnsdorff's heuristic)
        return [(row_new, col_new) for _, row_new, col_new in moves]

    def get_degree(self, visited, row, col):
        count = 0
        for index in range(len(self.path_row)):
            row_new = row + self.path_row[index]
            col_new = col + self.path_col[index]
            if self.if_valid_move(visited, row_new, col_new):
                count += 1
        return count

    def if_valid_move(self, visited, row_new, col_new):
        return (0 <= row_new < self.n) and (0 <= col_new < self.n) and (visited[row_new][col_new] == 0)

    def update_canvas(self, canvas, visited, cell_size):
        canvas.delete("all")
        for i in range(self.n):
            for j in range(self.n):
                color = "white" if (i + j) % 2 == 0 else "gray"
                canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill=color)
                if visited[i][j] > 0:
                    canvas.create_text(j * cell_size + cell_size // 2, i * cell_size + cell_size // 2, text=str(visited[i][j]), fill="red")
        for i in range(len(self.moves) - 1):
            start = self.moves[i]
            end = self.moves[i + 1]
            canvas.create_line(start[1] * cell_size + cell_size // 2, start[0] * cell_size + cell_size // 2, 
                               end[1] * cell_size + cell_size // 2, end[0] * cell_size + cell_size // 2, fill="blue", width=2)
        canvas.update()

class KnightTourApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Knight's Tour Solver")

        self.label = tk.Label(root, text="Welcome to the Knight's Tour Solver!", font=("Arial", 16))
        self.label.pack(pady=10)

        self.start_button = tk.Button(root, text="Start Knight's Tour", command=self.start_knight_tour, font=("Arial", 14))
        self.start_button.pack(pady=10)

    def start_knight_tour(self):
        try:
            n = simpledialog.askinteger("Input", "Enter the size of the chessboard (n x n):", minvalue=1, maxvalue=20)
            if n is None:
                return

            start_row = simpledialog.askinteger("Input", "Enter the starting row (0-indexed):", minvalue=0, maxvalue=n-1)
            if start_row is None:
                return

            start_col = simpledialog.askinteger("Input", "Enter the starting column (0-indexed):", minvalue=0, maxvalue=n-1)
            if start_col is None:
                return

            knight_tour = KnightTour(n)
            visited = [[0 for _ in range(n)] for _ in range(n)]  # Initialize the board

            visited[start_row][start_col] = 1  # Mark starting position
            knight_tour.moves.append((start_row, start_col))

            board_window = tk.Toplevel(self.root)
            board_window.title("Knight's Tour Visualization")

            cell_size = 600 // n
            canvas = tk.Canvas(board_window, width=n * cell_size, height=n * cell_size)
            canvas.pack()

            if not knight_tour.find_knight_tour(visited, start_row, start_col, 2, canvas, cell_size):
                messagebox.showinfo("Result", "No solution exists for the given board size and starting position.")
            else:
                messagebox.showinfo("Result", "Solution found! Check the visualization.")

        except ValueError as e:
            messagebox.showerror("Error", f"Invalid input: {e}")
        except Exception as e:
            messagebox.showerror("Error", f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = KnightTourApp(root)
    root.mainloop()

